# POC of hyperparameter tuning with Optuna on Harmony

In [1]:
import os
import sys
import pickle
import pandas as pd
import optuna
import numpy as np
import jax
from harmonypy import run_harmony

import anndata as ad
from scib_metrics.benchmark import Benchmarker, BioConservation, BatchCorrection

/home/icb/tim.treis/miniconda/envs/scverse/envs/batchcp_correct_harmony/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
data = pd.read_parquet("/home/icb/tim.treis/projects/broad_integrate/2023_Arevalo_BatchCorrection/outputs/scenario_7/raw.parquet")
data

,Cells_AreaShape_Area,Cells_AreaShape_BoundingBoxArea,Cells_AreaShape_BoundingBoxMaximum_X,Cells_AreaShape_BoundingBoxMaximum_Y,Cells_AreaShape_BoundingBoxMinimum_X,Cells_AreaShape_BoundingBoxMinimum_Y,Cells_AreaShape_Center_X,Cells_AreaShape_Center_Y,Cells_AreaShape_Compactness,Cells_AreaShape_Eccentricity,...,Metadata_Well,Metadata_JCP2022,Metadata_InChIKey,Metadata_InChI,Metadata_Batch,Metadata_PlateType,Metadata_PertType,Metadata_Row,Metadata_Column,Metadata_Microscope
0,3846.399902,8135.600098,594.880005,544.299988,506.290009,454.049988,550.309998,498.750000,2.8704,0.80747,...,A01,JCP2022_043547,KBPLFHHGFOOTCA-UHFFFAOYSA-N,"InChI=1S/C8H18O/c1-2-3-4-5-6-7-8-9/h9H,2-8H2,1H3",2021_08_03_U2OS_48_hr_run12,TARGET2,trt,A,01,CV8000
1,3484.199951,7088.100098,541.400024,539.609985,458.500000,455.570007,499.720001,496.950012,2.6773,0.78594,...,A02,Quinidine,LOUPRKONTZGTKE-UHFFFAOYSA-N,InChI=1S/C20H24N2O2/c1-3-13-12-22-9-7-14(13)10...,2021_08_03_U2OS_48_hr_run12,TARGET2,poscon,A,02,CV8000
2,3281.899902,6740.299805,540.909973,564.090027,458.880005,483.440002,498.920013,523.299988,2.6837,0.79269,...,A03,JCP2022_050997,LPYXWGMUVRGUOY-UHFFFAOYSA-N,InChI=1S/C6H8O6/c7-1-2(8)5-3(9)4(10)6(11)12-5/...,2021_08_03_U2OS_48_hr_run12,TARGET2,trt,A,03,CV8000
3,3476.500000,7233.500000,554.900024,491.649994,472.109985,406.529999,512.919983,448.769989,2.7987,0.79305,...,A04,JCP2022_108326,YGSDEFSMJLZEOE-UHFFFAOYSA-N,"InChI=1S/C7H6O3/c8-6-4-2-1-3-5(6)7(9)10/h1-4,8...",2021_08_03_U2OS_48_hr_run12,TARGET2,trt,A,04,CV8000
4,3592.100098,7581.799805,565.460022,555.419983,480.459991,468.079987,522.429993,511.429993,2.9000,0.79549,...,A05,DMSO,IAZDPXIOMUYVGZ-UHFFFAOYSA-N,InChI=1S/C2H6OS/c1-4(2)3/h1-2H3,2021_08_03_U2OS_48_hr_run12,TARGET2,negcon,A,05,CV8000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25324,3332.300049,6904.899902,537.479980,563.710022,455.929993,480.660004,496.239990,521.619995,2.7502,0.79097,...,P20,JCP2022_032357,HSUGRBWQSSZJOP-UHFFFAOYSA-N,InChI=1S/C22H26N2O4S/c1-15(25)28-20-21(16-9-11...,J2,TARGET2,trt,P,20,ImageXpress Micro Confocal
25325,2435.600098,5024.700195,545.000000,542.119995,475.519989,471.579987,509.600006,506.649994,2.4488,0.79012,...,P21,JCP2022_047545,KXBDTLQSDKGAEB-UHFFFAOYSA-N,InChI=1S/C22H22FN5O3/c1-3-20(29)25-16-5-4-6-17...,J2,TARGET2,trt,P,21,ImageXpress Micro Confocal
25326,3204.000000,6467.200195,551.619995,557.239990,471.690002,478.299988,510.970001,517.450012,2.6087,0.77934,...,P22,JCP2022_043099,JZFPYUNJRRFVQU-UHFFFAOYSA-N,"InChI=1S/C13H9F3N2O2/c14-13(15,16)8-3-1-4-9(7-...",J2,TARGET2,trt,P,22,ImageXpress Micro Confocal
25327,2613.899902,5493.399902,553.880005,540.500000,480.779999,467.940002,516.609985,503.779999,2.6514,0.78933,...,P23,JCP2022_050516,LNFZRMDSZJCZTG-UHFFFAOYSA-N,InChI=1S/C28H21F4N5O2/c1-15-4-6-19(35-27(39)37...,J2,TARGET2,trt,P,23,ImageXpress Micro Confocal


In [2]:
adata = ad.read("/home/icb/tim.treis/projects/broad_integrate/2023_Arevalo_BatchCorrection/outputs/scenario_7/mad_int_featselect_all_methods.h5ad")
adata

/home/icb/tim.treis/miniconda/envs/scverse/envs/batchcp_correct_harmony/lib/python3.13/site-packages/anndata/__init__.py:52: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(


AnnData object with n_obs × n_vars = 25329 × 1018
    obs: 'Metadata_Source', 'Metadata_Plate', 'Metadata_Well', 'Metadata_JCP2022', 'Metadata_InChIKey', 'Metadata_InChI', 'Metadata_Batch', 'Metadata_PlateType', 'Metadata_PertType', 'Metadata_Row', 'Metadata_Column', 'Metadata_Microscope'
    uns: 'ot_cfm_embeddings', 'pca'
    obsm: 'combat', 'cpDistiller', 'cpDistiller_B', 'cpDistiller_S', 'cpDistiller_SBP', 'desc', 'fastMNN', 'gaushanvi', 'gaushvi', 'harmony', 'ot_cfm', 'scanorama', 'scanorama_pca', 'scanvi', 'scpoli', 'scvi', 'seurat_cca', 'seurat_rpca', 'sphering', 'sysvi'
    varm: 'PCs'

In [3]:
for pkg in [jax]:
    print(f'{pkg.__name__}: {pkg.__version__}')

jax: 0.5.2


## Load data

In [2]:
data_path = "/home/icb/tim.treis/projects/broad_integrate/2023_Arevalo_BatchCorrection/outputs/scenario_7/mad_int_featselect.parquet"

data = pd.read_parquet(data_path)
metadata_cols = data.filter(regex="Metadata").columns

adata = ad.AnnData(X=data.drop(metadata_cols, axis=1).values, obs=data[metadata_cols])
adata

/home/icb/tim.treis/miniconda/envs/scverse/envs/batchcp_correct_harmony/lib/python3.13/site-packages/anndata/_core/aligned_df.py:68: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


AnnData object with n_obs × n_vars = 25329 × 1018
    obs: 'Metadata_Source', 'Metadata_Plate', 'Metadata_Well', 'Metadata_JCP2022', 'Metadata_InChIKey', 'Metadata_InChI', 'Metadata_Batch', 'Metadata_PlateType', 'Metadata_PertType', 'Metadata_Row', 'Metadata_Column', 'Metadata_Microscope'

## Define Optuna trials and evaluation logic

In [4]:
def scib_benchmark_embedding(
    adata: ad.AnnData,
    batch_key: str,
    label_key: str,
) -> float:
    adata.obsm["trial"] = adata.X

    # silence output
    sys.stdout = open(os.devnull, "w")

    bm = Benchmarker(
        adata=adata,
        batch_key=batch_key,
        label_key=label_key,
        embedding_obsm_keys=["trial"],
        bio_conservation_metrics=BioConservation(),
        batch_correction_metrics=BatchCorrection(),
    )
    bm.benchmark()
    df = bm.get_results(min_max_scale=False)

    # restore output
    sys.stdout.close()
    sys.stdout = sys.__stdout__

    return df.loc["trial"][["Batch correction", "Bio conservation"]].values


def objective(
    trial,
    adata: ad.AnnData, 
    batch_key: str, 
    label_key: str, 
    smoketest: bool = False,
):
    # Silence output
    sys.stdout = open(os.devnull, "w")

    # Optimize Harmony-specific hyperparameters
    sigma  = trial.suggest_float("sigma", 0.05, 1.0)
    theta  = trial.suggest_float("theta", 0.1, 5.0)
    lamb   = trial.suggest_float("lamb", 0.1, 5.0)
    nclust = trial.suggest_int("nclust", 2, 500)
    tau    = trial.suggest_float("tau", 0.0, 1.0)

    # Ensure the data matrix is a dense numpy array
    if not isinstance(adata.X, np.ndarray):
        data_mat = adata.X.toarray()
    else:
        data_mat = adata.X
    meta_data = adata.obs.copy()

    ho = run_harmony(
        data_mat=data_mat,
        meta_data=meta_data,
        vars_use=batch_key,
        theta=theta,
        lamb=lamb,
        sigma=sigma,
        nclust=nclust,
        tau=tau,
        random_state=0
    )

    # Retrieve the integrated data (Z_corr has shape [d, N]) and transpose to [N, d]
    vals = ho.result().T
    features = [f"harmony_{i}" for i in range(vals.shape[1])]
    integrated_adata = ad.AnnData(
        X=pd.DataFrame(vals, columns=features, index=adata.obs_names),
        obs=adata.obs.copy()
    )

    # Evaluate the integration using scIB metrics
    batch, bio = scib_benchmark_embedding(
        adata=integrated_adata,
        batch_key=batch_key, 
        label_key=label_key
    )

    # Restore output
    sys.stdout.close()
    sys.stdout = sys.__stdout__

    return batch, bio

batch_key = "Metadata_Source"
label_key = "Metadata_JCP2022"

study = optuna.create_study(directions=["maximize", "maximize"])
study.optimize(lambda trial: objective(trial, adata.copy(), batch_key, label_key), n_trials=1)

[I 2025-03-23 18:55:33,192] A new study created in memory with name: no-name-753fc7ae-02e6-4435-8c7c-1833d1bf2d9c
2025-03-23 18:55:33,740 - harmonypy - INFO - Computing initial centroids with sklearn.KMeans...
2025-03-23 18:59:08,913 - harmonypy - INFO - sklearn.KMeans initialization complete.
2025-03-23 18:59:09,400 - harmonypy - INFO - Iteration 1 of 10
2025-03-23 19:01:04,359 - harmonypy - INFO - Iteration 2 of 10
2025-03-23 19:01:52,405 - harmonypy - INFO - Iteration 3 of 10
2025-03-23 19:03:01,379 - harmonypy - INFO - Iteration 4 of 10
2025-03-23 19:04:24,065 - harmonypy - INFO - Converged after 4 iterations
/home/icb/tim.treis/miniconda/envs/scverse/envs/batchcp_correct_harmony/lib/python3.13/site-packages/scanpy/preprocessing/_pca.py:374: FutureWarning: Argument `use_highly_variable` is deprecated, consider using the mask argument. Use_highly_variable=True can be called through mask_var="highly_variable". Use_highly_variable=False can be called through mask_var=None
  warn(msg, 

In [6]:
study.best_trials

[]

## POC works, run it as headless script now with enough trials

In [3]:
with open("./optuna_study.pkl", "rb") as input_file:
    study = pickle.load(input_file)

In [6]:
optuna.visualization.plot_pareto_front(study, target_names=["Batch", "Bio"])

### Convert to table

In [7]:
trials = study.trials

data = []
for trial in trials:
    batch, bio = trial.values
    sorted_params = {k: trial.params[k] for k in sorted(trial.params.keys())}

    row = {
        "state": trial.state.name,
        "batch": batch,
        "bio": bio,
        "total": 0.6 * bio + 0.4 * batch,
        **sorted_params,          # Hyperparameters
    }
    data.append(row)


data = pd.DataFrame(data)
data = data.sort_values("total", ascending=False)

# custom sort columns
custom_order = ["state", "batch", "bio", "total"]  # Columns to appear first
remaining_columns = sorted([col for col in data.columns if col not in custom_order])
final_column_order = custom_order + remaining_columns
data = data[final_column_order]

data

,state,batch,bio,total,alpha_epoch_anneal,embedding_dims,eta,latent_dim,layer_0_size,layer_1_size,layer_2_size,num_layers,pretrain_to_train_ratio
26,COMPLETE,0.369717,0.505063,0.450925,600,5,0.6,32,416,96.0,384.0,3,0.5
0,COMPLETE,0.348516,0.516480,0.449294,300,13,0.5,32,480,288.0,NaN,2,0.5
11,COMPLETE,0.331015,0.522050,0.445636,700,4,0.8,128,256,NaN,NaN,1,0.5
30,COMPLETE,0.349754,0.508287,0.444874,800,19,0.8,96,384,256.0,NaN,2,0.4
38,COMPLETE,0.333959,0.518329,0.444581,600,7,0.9,80,160,NaN,NaN,1,0.5
42,COMPLETE,0.350704,0.506698,0.444301,200,3,0.7,32,64,480.0,NaN,2,0.5
45,COMPLETE,0.352857,0.504641,0.443927,300,8,0.8,32,416,512.0,448.0,3,0.1
12,COMPLETE,0.341758,0.511121,0.443376,800,8,1.0,80,320,480.0,NaN,2,0.1
47,COMPLETE,0.337732,0.504015,0.437502,900,10,0.7,32,128,416.0,NaN,2,0.4
28,COMPLETE,0.330649,0.505752,0.435711,400,8,0.7,32,384,480.0,NaN,2,0.1


In [18]:
df = study.trials_dataframe()
df = df.rename(columns={"values_0": "batch", "values_1": "bio"})
df["total"] = 0.6 * df["bio"] + 0.4 * df["batch"]
df = df.sort_values("total", ascending=False)
df

,number,batch,bio,datetime_start,datetime_complete,duration,params_alpha_epoch_anneal,params_embedding_dims,params_eta,params_latent_dim,params_layer_0_size,params_layer_1_size,params_layer_2_size,params_num_layers,params_pretrain_to_train_ratio,system_attrs_nsga2:generation,state,total
26,26,0.369717,0.505063,2024-12-02 22:59:16.571127,2024-12-02 23:00:58.440468,0 days 00:01:41.869341,600,5,0.6,32,416,96.0,384.0,3,0.5,0,COMPLETE,0.450925
0,0,0.348516,0.516480,2024-12-02 21:18:13.889310,2024-12-02 21:21:52.457649,0 days 00:03:38.568339,300,13,0.5,32,480,288.0,NaN,2,0.5,0,COMPLETE,0.449294
11,11,0.331015,0.522050,2024-12-02 21:53:36.683358,2024-12-02 21:55:15.499466,0 days 00:01:38.816108,700,4,0.8,128,256,NaN,NaN,1,0.5,0,COMPLETE,0.445636
30,30,0.349754,0.508287,2024-12-02 23:16:30.105069,2024-12-02 23:20:15.210932,0 days 00:03:45.105863,800,19,0.8,96,384,256.0,NaN,2,0.4,0,COMPLETE,0.444874
38,38,0.333959,0.518329,2024-12-02 23:45:51.658629,2024-12-02 23:47:27.366824,0 days 00:01:35.708195,600,7,0.9,80,160,NaN,NaN,1,0.5,0,COMPLETE,0.444581
42,42,0.350704,0.506698,2024-12-03 00:00:56.923245,2024-12-03 00:02:37.577585,0 days 00:01:40.654340,200,3,0.7,32,64,480.0,NaN,2,0.5,0,COMPLETE,0.444301
45,45,0.352857,0.504641,2024-12-03 00:11:33.527766,2024-12-03 00:22:22.479232,0 days 00:10:48.951466,300,8,0.8,32,416,512.0,448.0,3,0.1,0,COMPLETE,0.443927
12,12,0.341758,0.511121,2024-12-02 21:55:15.500483,2024-12-02 22:05:23.076041,0 days 00:10:07.575558,800,8,1.0,80,320,480.0,NaN,2,0.1,0,COMPLETE,0.443376
47,47,0.337732,0.504015,2024-12-03 00:30:51.971004,2024-12-03 00:34:47.506981,0 days 00:03:55.535977,900,10,0.7,32,128,416.0,NaN,2,0.4,0,COMPLETE,0.437502
28,28,0.330649,0.505752,2024-12-02 23:04:29.309385,2024-12-02 23:15:14.648358,0 days 00:10:45.338973,400,8,0.7,32,384,480.0,NaN,2,0.1,0,COMPLETE,0.435711
